# Google Colab: Drive'daki ZIP'i çıkartma
Bu not defteri, Google Drive'da bulunan büyük ZIP arşivini güvenle çıkartmak için gerekli adımları içerir. Adımları sırayla çalıştırın; ZIP yolu ve hedef klasörü size göre güncelleyin.

In [ ]:
# 1) Gerekli kütüphaneler ve Drive bağlama
try:
    from pathlib import Path
    import os, subprocess
    print("Libraries imported.")
except Exception as e:
    print("Import error:", e)
    raise

try:
    from google.colab import drive  # type: ignore
    print("Drive bağlanıyor...")
    drive.mount('/content/drive', force_remount=True)
except Exception as e:
    print("Colab Drive modülü yok veya bağlanamadı:", e)

In [ ]:
# 2) ZIP ve hedef klasörleri tanımla
from pathlib import Path
ZIP_INPUT = Path('/content/drive/MyDrive/omr_dataset/dataset/ds2/ds2_complete_tmn/ds2_complete_tmn.zip')  # Burayı kendi ZIP yoluna göre ayarla
EXTRACT_DIR = Path('/content/ds2_complete_tmn')  # Çıkartma hedefi
EXTRACT_DIR.mkdir(parents=True, exist_ok=True)

print('ZIP_INPUT =', ZIP_INPUT)
print('EXTRACT_DIR =', EXTRACT_DIR)
print('ZIP mevcut mu?', ZIP_INPUT.exists())

In [ ]:
# 3) 7-Zip (p7zip) ve unrar kur; ZIP/RAR çıkar
!apt-get update -y >/dev/null 2>&1 && apt-get install -y p7zip-full unrar >/dev/null 2>&1

import subprocess
from pathlib import Path

if ZIP_INPUT.exists():
    ext = ZIP_INPUT.suffix.lower()
    if ext == '.zip':
        cmd = f'7z x "{ZIP_INPUT}" -o"{EXTRACT_DIR}" -y'
        print('ZIP tespit edildi, 7z ile çıkarılıyor...')
        print('CMD:', cmd)
        rc = subprocess.call(cmd, shell=True)
        if rc != 0:
            print('7z başarısız; unzip ile denenecek...')
            unzip_cmd = f'unzip -o "{ZIP_INPUT}" -d "{EXTRACT_DIR}"'
            print('CMD:', unzip_cmd)
            rc2 = subprocess.call(unzip_cmd, shell=True)
            print('unzip exit code:', rc2)
    elif ext == '.rar':
        print('RAR tespit edildi, unrar ile çıkarılıyor...')
        unrar_cmd = f'unrar x -o+ "{ZIP_INPUT}" "{EXTRACT_DIR}"'
        print('CMD:', unrar_cmd)
        rc = subprocess.call(unrar_cmd, shell=True)
        if rc != 0:
            print('unrar başarısız; 7z ile denenecek...')
            cmd = f'7z x "{ZIP_INPUT}" -o"{EXTRACT_DIR}" -y'
            print('CMD:', cmd)
            rc2 = subprocess.call(cmd, shell=True)
            print('7z exit code:', rc2)
    else:
        print(f'Bilinmeyen uzantı: {ext}. 7z ile denenecek...')
        cmd = f'7z x "{ZIP_INPUT}" -o"{EXTRACT_DIR}" -y'
        print('CMD:', cmd)
        rc = subprocess.call(cmd, shell=True)
        print('7z exit code:', rc)
else:
    print('ZIP_INPUT bulunamadı; yolu kontrol edin.')

In [ ]:
# 4) Çıkartılan içeriği doğrula ve örnek listeleme
import subprocess
print('Hızlı boyut özeti:')
subprocess.call(f"du -sh '{EXTRACT_DIR}'", shell=True)
subprocess.call(f"du -sh '{EXTRACT_DIR}'/* 2>/dev/null | sed -n '1,20p'", shell=True)

# İlk birkaç dosya
subprocess.call(f"find '{EXTRACT_DIR}' -maxdepth 2 -type f | head -n 20", shell=True)